# Spark Context

In [1]:
from pyspark.sql import SparkSession

# Spark session & context
spark = (SparkSession
         .builder
         .master("local")
         .appName("spark-sql")
         .getOrCreate())
sc = spark.sparkContext

24/08/25 13:59:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/25 13:59:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
df = (
    spark.read
    .format("csv")
    .option("header", True)
    .load("/home/jovyan/data/movies.csv")
)

In [3]:
df.summary()

DataFrame[summary: string, movieId: string, title: string, genres: string]

In [21]:
df.show(10, False)

+-------+----------------------------------+-------------------------------------------+
|movieId|title                             |genres                                     |
+-------+----------------------------------+-------------------------------------------+
|1      |Toy Story (1995)                  |Adventure|Animation|Children|Comedy|Fantasy|
|2      |Jumanji (1995)                    |Adventure|Children|Fantasy                 |
|3      |Grumpier Old Men (1995)           |Comedy|Romance                             |
|4      |Waiting to Exhale (1995)          |Comedy|Drama|Romance                       |
|5      |Father of the Bride Part II (1995)|Comedy                                     |
|6      |Heat (1995)                       |Action|Crime|Thriller                      |
|7      |Sabrina (1995)                    |Comedy|Romance                             |
|8      |Tom and Huck (1995)               |Adventure|Children                         |
|9      |Sudden Death

In [7]:
grouped_df = df.groupBy('genres').pivot('title')
grouped_df.show(3)

AttributeError: 'GroupedData' object has no attribute 'show'

In [ ]:
grouped_df.count().show(3)

In [10]:
from pyspark.sql import functions as F

grouped_df.agg(F.sum('some_column'), F.avg('another_column')).show(3)

24/08/25 14:04:24 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


AnalysisException: cannot resolve '`some_column`' given input columns: [genres, movieId, title];
'Pivot ArrayBuffer(genres#18), title#17: string, ["11'09""01 - September 11 (2002)", '71 (2014), 'Hellboy': The Seeds of Creation (2004), 'Round Midnight (1986), 'Salem's Lot (2004), 'Til There Was You (1997), 'Tis the Season for Love (2015), 'burbs, The (1989), 'night Mother (1986), (500) Days of Summer (2009), *batteries not included (1987), ...All the Marbles (1981), ...And Justice for All (1979), 00 Schneider - Jagd auf Nihil Baxter (1994), 1-900 (06) (1994), 10 (1979), 10 Cent Pistol (2015), 10 Cloverfield Lane (2016), 10 Items or Less (2006), 10 Things I Hate About You (1999), 10 Years (2011), 10,000 BC (2008), 100 Girls (2000), 100 Streets (2016), ... 9713 more fields], [sum('some_column), avg('another_column)]
+- Relation[movieId#16,title#17,genres#18] csv


In [4]:
df.printSchema()

root
 |-- movieId: string (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [11]:
# movies라는 임시테이블을 만든 것
df.createOrReplaceTempView("movies")

In [12]:
n_df = spark.sql("SELECT DISTINCT(genres) FROM movies")
n_df.show(30, False)

+-------------------------------------------------+
|genres                                           |
+-------------------------------------------------+
|Comedy|Horror|Thriller                           |
|Adventure|Sci-Fi|Thriller                        |
|Action|Adventure|Drama|Fantasy                   |
|Action|Drama|Horror                              |
|Action|Animation|Comedy|Sci-Fi                   |
|Animation|Children|Drama|Musical|Romance         |
|Action|Adventure|Drama                           |
|Adventure|Sci-Fi                                 |
|Documentary|Musical|IMAX                         |
|Adventure|Children|Fantasy|Sci-Fi|Thriller       |
|Adventure|Animation                              |
|Musical|Romance|War                              |
|Action|Adventure|Fantasy|Romance                 |
|Adventure|Children|Drama|Fantasy|IMAX            |
|Comedy|Crime|Horror|Thriller                     |
|Crime|Drama|Fantasy|Horror|Thriller              |
|Comedy|Myst

In [22]:
spark.sql("SELECT title FROM movies WHERE movieId > 100").show(10, False)

+------------------------------------------+
|title                                     |
+------------------------------------------+
|Bottle Rocket (1996)                      |
|Mr. Wrong (1996)                          |
|Unforgettable (1996)                      |
|Happy Gilmore (1996)                      |
|Bridges of Madison County, The (1995)     |
|Nobody Loves Me (Keiner liebt mich) (1994)|
|Muppet Treasure Island (1996)             |
|Catwalk (1996)                            |
|Braveheart (1995)                         |
|Taxi Driver (1976)                        |
+------------------------------------------+
only showing top 10 rows



In [30]:
spark.sql("""
    SELECT movieId, genres
    FROM movies
    GROUP BY genres, movieId"""
    ).where("movieId > 10"
    ).count()

9732

## RDD with partitions

In [13]:
rdd1 = sc.textFile("../data/movies.csv")
rdd2 = sc.textFile("../data/ratings.csv")

In [14]:

print(f"defaultMinPartitions: {sc.defaultMinPartitions}")
print(f"name: rdd1, count: {rdd1.count}, partitions: {rdd1.getNumPartitions()}")
print(f"name: rdd2, count: {rdd2.count}, partitions: {rdd2.getNumPartitions()}")


defaultMinPartitions: 1
name: rdd1, count: <bound method RDD.count of ../data/movies.csv MapPartitionsRDD[68] at textFile at NativeMethodAccessorImpl.java:0>, partitions: 1
name: rdd2, count: <bound method RDD.count of ../data/ratings.csv MapPartitionsRDD[70] at textFile at NativeMethodAccessorImpl.java:0>, partitions: 1


24/08/25 14:07:18 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB
24/08/25 14:12:59 WARN DAGScheduler: Broadcasting large task binary with size 3.7 MiB


## Spark SQL

In [3]:
github = spark.read.json("../data/*.json.gz")
github.printSchema()

root
 |-- actor: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- display_login: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- org: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- payload: struct (nullable = true)
 |    |-- action: string (nullable = true)
 |    |-- before: string (nullable = true)
 |    |-- comment: struct (nullable = true)
 |    |    |-- _links: struct (nullable = true)
 |    |    |    |-- html: struct (nullable = true)
 |    |    |    |    |-- href: string (nullable = true)
 |    |    |    |-- pull_request: struct (nul

24/08/24 11:05:06 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [23]:
github.show(10)

+--------------------+--------------------+-----------+----+--------------------+------+--------------------+-----------+
|               actor|          created_at|         id| org|             payload|public|                repo|       type|
+--------------------+--------------------+-----------+----+--------------------+------+--------------------+-----------+
|{"id":83618436,"l...|2024-05-19T14:00:00Z|38509489462|null|{"repository_id":...|  true|{"id":489717552,"...|  PushEvent|
|{"id":132692,"log...|2024-05-19T14:00:00Z|38509489467|null|{"repository_id":...|  true|{"id":791822148,"...|  PushEvent|
|{"id":78722358,"l...|2024-05-19T14:00:00Z|38509489469|null|{"repository_id":...|  true|{"id":802788764,"...|  PushEvent|
|{"id":41898282,"l...|2024-05-19T14:00:00Z|38509489476|null|{"repository_id":...|  true|{"id":400840924,"...|  PushEvent|
|{"id":41898282,"l...|2024-05-19T14:00:00Z|38509489485|null|{"repository_id":...|  true|{"id":492467823,"...|  PushEvent|
|{"id":97881751,"l...|20

In [33]:
github.show(1, False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+-----------+----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+--------------------------------------------------------------------------------------------------------------------+---------+
|actor                                                                                                                         

In [71]:
from pyspark.sql.types import StructType, StructField, StringType, LongType
import pyspark.sql.functions as F

actor_schema = StructType([
    StructField('login', StringType(), True),
    StructField('url', StringType(), True)
])

payload_schema = StructType([
    StructField('repository_id', LongType(), True),
    StructField('size', LongType(), True),
    StructField('distinct_size', LongType(), True),
    StructField('message', StringType(), True)
])

repo_schema = StructType([
    StructField('name', StringType(), True),
    StructField('url', StringType(), True)
])

new_df = github.withColumn('actor_json', F.from_json('actor', actor_schema)) \
               .select('created_at', 'id', 'payload', 'type', 'actor_json.*', 'repo')
new_df = new_df.withColumn('payload_json', F.from_json('payload', payload_schema)) \
               .select('login', 'url', 'created_at', 'id', 'payload_json.*', 'type', 'repo')
new_df = new_df.withColumn('repo_json', F.from_json('repo', repo_schema)) \
               .select('login', 'url', 'created_at', 'id', 'repository_id', 'size', 'distinct_size', 'message', 'type', 'repo_json.*')
new_df.show(10, False)

+-------------------+------------------------------------------------+--------------------+-----------+-------------+----+-------------+-------+-----------+----------------------------------------+---------------------------------------------------------------------+
|login              |url                                             |created_at          |id         |repository_id|size|distinct_size|message|type       |name                                    |url                                                                  |
+-------------------+------------------------------------------------+--------------------+-----------+-------------+----+-------------+-------+-----------+----------------------------------------+---------------------------------------------------------------------+
|Drunkula           |https://api.github.com/users/Drunkula           |2024-05-19T14:00:00Z|38509489462|489717552    |1   |1            |null   |PushEvent  |Drunkula/twitchtoolsglitch              


|login     |url    |created_at    |id   |repository_id|size|distinct_size|message|type  |
| --- | ---| ---| --- | --- | --- | --- | --- | --- |
| String | String | DateTime | Long | Long | Int | Int | String | String |

Top 50 Push repositories
Top 50 Commit repositories



In [55]:
new_df.printSchema()

root
 |-- login: string (nullable = true)
 |-- url: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- repository_id: long (nullable = true)
 |-- size: long (nullable = true)
 |-- distinct_size: long (nullable = true)
 |-- message: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- url: string (nullable = true)



In [72]:
# filter github action bot
new_df = new_df.filter(col("login") != "github-actions[bot]")
new_df = new_df.withColumn('created_at', F.trim(F.regexp_replace(new_df.created_at, "[TZ]", " ")))
new_df = new_df.withColumn('created_dt', F.to_timestamp(new_df.created_at, 'yyyy-MM-dd HH:mm:ss'))
new_df.show(10, False)

+------------+-----------------------------------------+-------------------+-----------+-------------+----+-------------+-------+-----------+----------------------------------------+---------------------------------------------------------------------+-------------------+
|login       |url                                      |created_at         |id         |repository_id|size|distinct_size|message|type       |name                                    |url                                                                  |created_dt         |
+------------+-----------------------------------------+-------------------+-----------+-------------+----+-------------+-------+-----------+----------------------------------------+---------------------------------------------------------------------+-------------------+
|Drunkula    |https://api.github.com/users/Drunkula    |2024-05-19 14:00:00|38509489462|489717552    |1   |1            |null   |PushEvent  |Drunkula/twitchtoolsglitch              

In [73]:
new_df.printSchema()

root
 |-- login: string (nullable = true)
 |-- url: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- repository_id: long (nullable = true)
 |-- size: long (nullable = true)
 |-- distinct_size: long (nullable = true)
 |-- message: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- url: string (nullable = true)
 |-- created_dt: timestamp (nullable = true)



In [64]:
def check_repo_name(name):
    sp = name.split("/")
    if not sp:
        return name
    else:
        return sp[-1]
    
udf_check_repo_name = F.udf(check_repo_name, StringType())

In [ ]:
# @F.udf(returnType=StringType())
# def check_repo_name(val):
#     sp = name.split("/")
#     if not sp:
#         return name
#     else:
#         return sp[-1]

In [ ]:
# F.udf(lambda name => name.split("/")[-1], StringType())

In [83]:
new_df = new_df.withColumn('repo_name', udf_check_repo_name(F.col('name')))
new_df.show(10, False)

+------------+-----------------------------------------+-------------------+-----------+-------------+----+-------------+-------+-----------+----------------------------------------+---------------------------------------------------------------------+-------------------+---------------------------------+
|login       |url                                      |created_at         |id         |repository_id|size|distinct_size|message|type       |name                                    |url                                                                  |created_dt         |repo_name                        |
+------------+-----------------------------------------+-------------------+-----------+-------------+----+-------------+-------+-----------+----------------------------------------+---------------------------------------------------------------------+-------------------+---------------------------------+
|Drunkula    |https://api.github.com/users/Drunkula    |2024-05-19 14:00:00|385

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


In [85]:
new_df.count()

171662

In [84]:
new_df.agg(F.countDistinct('repo_name')).show(5, False)

+----------------+
|count(repo_name)|
+----------------+
|50439           |
+----------------+



In [88]:
# Window
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number

w = Window.partitionBy("repo_name").orderBy(F.desc(col("size")))
new_df.withColumn("row", row_number().over(w)) \
      .filter(col("row") == 1).drop("row") \
      .count()

50439

## Data Model

1. Top 10 Repo
- id
- @timestamp
- repo_url
- repo_name
- push_count
- commit_count
- pr_count
- fork_count
- issue_count
- watch_count

2. Top 10 User
- id
- @timestamp
- user_name
- push_count
- commit_count
- pr_count
- issue_count
- issue_comment_count

3. Daily Stats
- id
- @timestamp
- distinct_user_cnt
- distinct_repo_cnt
- push_count
- commit_count
- pr_count
- issue_count
- issue_comment_count
- release_count

In [89]:
new_df.select("type").distinct().show()

+--------------------+
|                type|
+--------------------+
|PullRequestReview...|
|           PushEvent|
|         GollumEvent|
|        ReleaseEvent|
|  CommitCommentEvent|
|         CreateEvent|
|PullRequestReview...|
|   IssueCommentEvent|
|         DeleteEvent|
|         IssuesEvent|
|           ForkEvent|
|         PublicEvent|
|         MemberEvent|
|          WatchEvent|
|    PullRequestEvent|
+--------------------+

